# Анализ факторов, влияющих на успеваемость студентов

## Подготовка к работе

In [1]:
# Импорт необходимых для работы библиотек

import pandas as pd
from scipy import stats as st

In [2]:
# Отключение предупреждений в итоговом отчёте

import warnings
warnings.filterwarnings('ignore')

In [3]:
# Выгрузка предобработанной версии датасета `jamb` 

jamb = pd.read_csv('/Users/mimikhailova/ВШЭ/jamb.csv')

In [4]:
jamb.head()

,jamb_score,study_hours_per_week,attendance_rate,teacher_quality,distance_to_school,school_type,school_location,extra_tutorials,access_to_learning_materials,parent_involvement,it_knowledge,age,gender,socioeconomic_status,parent_education_level,assignments_completed
0,192,22,78,4,12.4,Public,Urban,Yes,Yes,High,Medium,17,Male,Low,Tertiary,2
1,207,14,88,4,2.7,Public,Rural,No,Yes,High,High,15,Male,High,Unknown,1
2,182,29,87,2,9.6,Public,Rural,Yes,Yes,High,Medium,20,Female,High,Tertiary,2
3,210,29,99,2,2.6,Public,Urban,No,Yes,Medium,High,22,Female,Medium,Tertiary,1
4,199,12,98,3,8.8,Public,Urban,No,Yes,Medium,Medium,22,Female,Medium,Tertiary,1


## Проверка гипотез

### Верно ли, что студенты, проживающие ближе к месту обучения, сдают экзамены лучше?

**Предпосылки:**  студенты, которые живут далеко от школы, начинают свой день раньше остальных. Не исключено, что из-за этого они меньше спят и соответственно меньше отдыхают. Утренняя дорога - это нагрузка на организм, которая приводит к снижению уровня энергии, с которой студент начинает учиться в школе. Из-за этого он может быть менее сконцентрирован и вовлечен в процесс, а значит может и хуже усваивать материал. 

Медианное значение расстояния равно 10.0 км, то есть ровно половина студентов проживает на расстоянии менее 10 км от школы, а ровно половина – более. Сделаем срезы на основании этого значения:

In [5]:
distance_less = jamb[jamb['distance_to_school'] <= 10]
distance_more = jamb[jamb['distance_to_school'] > 10]

Чтобы сформулировать гипотезы и выбрать нужный тест, проверим данные на нормальность с помощью теста Шапиро-Уилка:

In [6]:
# Тест Шапиро-Уилка 

shapiro_less = st.shapiro(distance_less['jamb_score'])
print(f"Для расстояния <=10 км: statistic = {shapiro_less.statistic}, p-value = {shapiro_less.pvalue}")

shapiro_more = st.shapiro(distance_more['jamb_score'])
print(f"Для расстояния >10 км:statistic = {shapiro_more.statistic}, p-value = {shapiro_more.pvalue}")

alpha = 0.05

if shapiro_less.pvalue < alpha:
    print("Данные в группе c расстоянием не более 10 км до школы не распределены нормально")
else:
    print("Данные в группе c расстоянием не более 10 км до школы распределены нормально")
    
if shapiro_more.pvalue < alpha:
    print("Данные в группе c расстоянием более 10 км до школы не распределены нормально")
else:
    print("Данные в группе c расстоянием более 10 км до школы распределены нормально")

Для расстояния <=10 км: statistic = 0.9704516530036926, p-value = 1.7256705906234346e-22
Для расстояния >10 км:statistic = 0.9645575284957886, p-value = 4.214185280264293e-24
Данные в группе c расстоянием не более 10 км до школы не распределены нормально
Данные в группе c расстоянием более 10 км до школы не распределены нормально


Видим, что данные обеих групп не распределены нормально, поэтому использовать t-тест нежелательно. 

В таком случае воспользуемся **тестом Манна-Уитни** для сравнения двух независимых групп. Сформулируем гипотезы:

**Нулевая гипотеза Н0:** распределения баллов JAMB у студентов, проживающих ≤10 км и >10 км от школы, одинаковы.

**Альтернативная правосторонняя гипотеза Н1**: у студентов, которые живут ближе к школе (≤10 км), распределение баллов за экзамен смещено в сторону более высоких значений по сравнению с распределением баллов у студентов, проживающих >10 км от школы.

In [7]:
# Тест Манна-Уитни для двух независимых выборок

alpha = 0.05

result = st.mannwhitneyu(distance_less['jamb_score'], distance_more['jamb_score'], alternative='greater')

print(f"U-статистика: {result.statistic:.2f}, p-значение: {result.pvalue}")
  
if result.pvalue < alpha:
      print('Отвергаем нулевую гипотезу')
else:
      print('Не получилось отвергнуть нулевую гипотезу')

U-статистика: 3390343.50, p-значение: 8.99400123779393e-08
Отвергаем нулевую гипотезу


📌 **Выводы:**
* Результаты теста Манна-Уитни показали статистически значимое различие между баллами JAMB студентов, проживающих на расстоянии менее или равном 10 км от школы, и студентов, проживающих дальше. Р-значение значительно меньше уровня значимости, поэтому можно отвергнуть нулевую гипотезу. Баллы студентов, живущих ближе к школе, статистически выше, чем баллы студентов, живущих на большем расстоянии.